# Bluesky Device for APS taxi & fly scans

Some EPICS fly scans at APS are triggered by a pair of EPICS
[*busy*](https://epics-modules.github.io/busy/) records.  The first *busy*
record is called `taxi` and is responsible for preparing the hardware to fly.
Once *taxi* is complete, the second *busy* record called `fly` performs the actual fly scan.
In a third (optional) phase, data is collected from hardware and written to a
file.

Compare the taxi & fly scan algorithm to an airplane flight:

phase | airplane flight | taxi & fly scan
--- | --- | ---
preparation | ticketing & boarding | configuration of software
taxi | move the aircraft to the start of the runway | move the hardware to pre-scan positions
fly | start moving, liftoff at flight velocity | start moving, begin collecting data at first position

## Bluesky (Python)setup

Here are the packages to import.  The first block contains Python standard
packages, then come the various bluesky packages.  Just the parts we plan on
using here.

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the bluesky `RunEngine`.
* Create a temporary databroker catalog to save collected data.
* Subscribe the catalog to receive all data published by the RunEngine.

In [1]:
import logging

from apstools.synApps import SseqRecord
from apstools.plans import run_blocking_function
import bluesky
import bluesky.plan_stubs as bps
import databroker
from ophyd import Component, Device, EpicsSignal

logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,  # more details than default (WARNING) level
    format="%(asctime)s %(levelname)s %(name)s - %(message)s"
)

RE = bluesky.RunEngine()
cat = databroker.temp().v2
RE.subscribe(cat.v1.insert)
# RE.msg_hook = bluesky.utils.ts_msg_hook  # deeper logging details

0

## EPICS IOC

We'll start with an EPICS IOC that provides two instances of the
[*busy*](https://epics-modules.github.io/busy/) record and two instances of the
[*sseq*](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/calc/R3-6-1/documentation/sseqRecord.html)
record.

In the `gp:` IOC, we can use these general purpose PVs for this example:

PV | record | purpose
--- | --- | ---
`gp:mybusy1` | *busy* | taxi (preparation) phase
`gp:mybusy2` | *fly* | fly (fly scan) phase
`gp:userStringSeq1` | *sseq* | actions for the taxi phase
`gp:userStringSeq2` | *sseq* | actions for the fly phase

The EPICS _busy_ record is used to start a phase (taxi or fly) and report when that
phase is finished. The EPICS _sseq_ record describes and manages the work to be done
in that phase. In this example, the work is simple (wait for some seconds). In
other situations, the work could involve a complex setup of software to collect data
or a complete
[_sscan_](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) process.

We need to configure these records for our use. We'll use the ophyd support for
[sseq](https://bcda-aps.github.io/apstools/latest/api/synApps/_sseq.html)
records from [apstools](https://bcda-aps.github.io/apstools). For the _busy_
records, we choose a simpler connections than the
[busy](https://bcda-aps.github.io/apstools/latest/api/synApps/_busy.html)
support from apstools. Instead, we'll connect with the `.OUT` and `.VAL` fields
using `EpicsSignal` (from ophyd). This allows us to add the `put_complete=True`
keyword argument that we need for the `.VAL` field's connection. The
`put_complete` keyword provides prompt notification when the asynchronous _busy_
record is finished.

We'll connect ophyd structures with these PVs and make bluesky plans here to
setup the PVs for this demo and to run the taxi+fly scan.

**NOTE**: This example does not collect any data. It only shows how the taxi &
fly scan is operated from bluesky.

### EPICS: How do the *busy* and *sseq* records interact in this demo?

Summary: The *busy* record tells the *sseq* record to do all its processing
steps.  The *sseq* record waits its assigned time, then turns the *busy* record
off.

Details:
The user changes the *busy* record's `.VAL` field from 0 to 1.  This causes the
*busy* record to send a 1 to the PV named in its `.OUT` field.  That field
contains the *sseq* record's `.PROC` field.  Thus, it causes the *sseq* record
to begin its processing steps.

There are two steps for the *sseq* record:

1. Wait for the assigned time.
2. Tell the *busy* record it is done.

One additional action was needed, to remove the PV name from the *busy* record's
`.OUT` field.  Without this additional action, the *sseq* record will be
processed by every change to the *busy* record value.

All of this configuration is programmed in the `setup_plan()` in the
`TaxiFlyScanDevice` class below.

### Call a blocking function from a bluesky plan

The `SseqRecord()` support in apstools has a `reset()` method to clear any
previous settings of the EPICS PVs and ophyd object and return them to default
settings.  This method is written as ophyd code, intended to be called from a
command-line session.  The commands it contains that may take some time to
complete and possibly block the normal execution of the RunEngine's callback
thread. The
[run_blocking_function()](https://bcda-aps.github.io/apstools/latest/api/_plans.html#module-apstools.plans.run_blocking_function_plan)
plan from *apstools* allows us to run `reset()` in a thread so that it does not
block the `RunEngine`.


## Define the TaxiFlyScanDevice

All of the taxi and fly components are configuration details and should not be included with any data to be recorded.  They are all marked with the `kind="config"` keyword.

In [2]:
class TaxiFlyScanDevice(Device):
    taxi = Component(EpicsSignal, "mybusy1", put_complete=True, kind="config")
    taxi_out = Component(EpicsSignal, "mybusy1.OUT", kind="config")
    taxi_sseq = Component(SseqRecord, "userStringSeq1", kind="config")

    fly = Component(EpicsSignal, "mybusy2", put_complete=True, kind="config")
    fly_out = Component(EpicsSignal, "mybusy2.OUT", kind="config")
    fly_sseq = Component(SseqRecord, "userStringSeq2", kind="config")

    def setup_plan(self, t_taxi, t_fly):
        logger.info("taxi time: %.2f s", t_taxi)
        logger.info("fly time: %.2f s", t_fly)
        yield from bps.mv(
            self.taxi, 0,
            self.fly, 0,
        )
        yield from bps.mv(
            self.taxi_out, f"{self.taxi_sseq.prefix}.PROC CA NMS",
            self.fly_out, f"{self.fly_sseq.prefix}.PROC CA NMS",
        )

        # Clear any previous setup of the sseq records.
        # The reset function is not a plan.
        yield from run_blocking_function(self.taxi_sseq.reset)
        yield from run_blocking_function(self.fly_sseq.reset)

        yield from bps.mv(
            # Set the delay time.
            self.taxi_sseq.steps.step1.delay, t_taxi,
            # Remove the .PROC poke so the sseq will not be triggered again.
            self.taxi_sseq.steps.step1.string_value, "",
            self.taxi_sseq.steps.step1.output_pv, f"{self.taxi_out.pvname} CA NMS",
            # Mark the busy record as Done.
            self.taxi_sseq.steps.step2.string_value, self.taxi.enum_strs[0],
            self.taxi_sseq.steps.step2.output_pv, f"{self.taxi.pvname} CA NMS",

            # Similar setup for the fly.
            self.fly_sseq.steps.step1.delay, t_fly,
            self.fly_sseq.steps.step1.string_value, "",
            self.fly_sseq.steps.step1.output_pv, f"{self.fly_out.pvname} CA NMS",
            self.fly_sseq.steps.step2.string_value, self.fly.enum_strs[0],
            self.fly_sseq.steps.step2.output_pv, f"{self.fly.pvname} CA NMS",
        )

    def taxi_fly_plan(self, time_taxi=5, time_fly=5):
        logger.info("before setup_plan")
        yield from self.setup_plan(time_taxi, time_fly)

        logger.info("before taxi")
        yield from bluesky.plan_stubs.mv(self.taxi, self.taxi.enum_strs[1])
        logger.info("after taxi")

        logger.info("before fly")
        yield from bluesky.plan_stubs.mv(self.fly, self.fly.enum_strs[1])
        logger.info("after fly")

Most of these lines are familiar ophyd or bluesky code.  Two lines might require more explanation:

```py
        # Clear any previous setup of the sseq records.
        # The reset function is not a plan.
        yield from run_blocking_function(self.taxi_sseq.reset)
        yield from run_blocking_function(self.fly_sseq.reset)
```

The `reset()` method from the `apstools.synApps.SseqRecord` is written as ophyd code.  This means that it calls EPICS directly.  Such code, when used in bluesky plans, will block the execution of bluesky's `RunEngine` while the main Python thread waits for the actions to complete.  Instead, bluesky plans should execute such actions in a background thread and surrender the waiting time to be managed by the `RunEngine`.

A *blocking function* (such as `SseqRecord`'s `reset()` method) can be used in a bluesky plan using the `run_blocking_function()` plan from `apstools.plans`.

## Create a Python object.

Here, the PVs are from the `gp:` IOC.

In [3]:
ifly = TaxiFlyScanDevice("gp:", name="ifly")
ifly.wait_for_connection()

## Run the complete plan

Run the `taxi_fly_plan()` with the bluesky RunEngine. There is no data to be
collected in this example, only logging entries.  Wait 2 seconds in the taxi
phase, then 3 seconds in the fly phase.

The call to `RE()` will return the list of _any_ run identifiers (`uid`s) that were created.  Grab that list.

In [4]:
uids = RE(ifly.taxi_fly_plan(2, 3))

2023-12-24 10:52:46,938 INFO bluesky - Executing plan <generator object TaxiFlyScanDevice.taxi_fly_plan at 0x7f10389ea7a0>
2023-12-24 10:52:47,055 INFO bluesky.RE.state - Change state on <bluesky.run_engine.RunEngine object at 0x7f0f88009850> from 'idle' -> 'running'
2023-12-24 10:52:47,056 INFO root - before setup_plan
2023-12-24 10:52:47,057 INFO root - taxi time: 2.00 s
2023-12-24 10:52:47,058 INFO root - fly time: 3.00 s
2023-12-24 10:52:47,125 INFO root - before taxi
2023-12-24 10:52:49,122 INFO root - after taxi
2023-12-24 10:52:49,124 INFO root - before fly
2023-12-24 10:52:52,122 INFO root - after fly
2023-12-24 10:52:52,123 INFO bluesky.RE.state - Change state on <bluesky.run_engine.RunEngine object at 0x7f0f88009850> from 'running' -> 'idle'
2023-12-24 10:52:52,124 INFO bluesky - Cleaned up from plan <generator object TaxiFlyScanDevice.taxi_fly_plan at 0x7f10389ea7a0>


There is no data to be collected.  Just print the list of `uid`s.  It should be empty in this instance.

In [5]:
print(f"{uids=}")

uids=()


## Fly scan with data

The previous example demonstrated the steps to use the APS Taxi & Fly support for a bluesky *fly* scan.  Now, let us add data collection to the example.

### Simulated temperature controller

We can simulate a [temperature controller](https://bcda-aps.github.io/bluesky_training/instrument/describe_instrument.html#temperature) with EPICS and Bluesky.  In EPICS, we simulate the controller using a [`swait`](https://bcda-aps.github.io/apstools/1.6.17/api/synApps/_swait.html) record.  Connect with `gp:userCalc18` from our example IOC.

In [45]:
from apstools.synApps import SwaitRecord

controller = SwaitRecord("gp:userCalc18", name="simulator")
controller.wait_for_connection()
print(f"{controller.read()=}\n{controller.read_configuration()=}")

controller.read()=OrderedDict([('simulator_calculated_value', {'value': 0.0, 'timestamp': 1703440261.074518})])
controller.read_configuration()=OrderedDict([('simulator_description', {'value': 'temperature', 'timestamp': 1703440261.07594}), ('simulator_scanning_rate', {'value': 10, 'timestamp': 1703440261.07594}), ('simulator_disable_value', {'value': 0, 'timestamp': 631152000.0}), ('simulator_scan_disable_input_link_value', {'value': 0, 'timestamp': 631152000.0}), ('simulator_scan_disable_value_input_link', {'value': 'gp:userCalcEnable.VAL CA MS', 'timestamp': 1703440261.07594}), ('simulator_forward_link', {'value': '0', 'timestamp': 1703440261.07594}), ('simulator_device_type', {'value': 0, 'timestamp': 631152000.0}), ('simulator_alarm_status', {'value': 1, 'timestamp': 1703440261.074518}), ('simulator_alarm_severity', {'value': 3, 'timestamp': 1703440261.074518}), ('simulator_new_alarm_status', {'value': 0, 'timestamp': 631152000.0}), ('simulator_new_alarm_severity', {'value': 0, 't

Create a function to configure the swait record as a simulated temperature controller.  The "controller" will update the current computed value (`.VAL`) at `rate` based on the setpoint (`.B`).  It will compute step size based on the difference between the previous value (`.A`) and the setpoint, taking steps no larger than `.D`.  A noise factor (`.C`) is applied to each new computation.

In [52]:
def setup_temperature(swait, setpoint=None, noise=2, rate=".5 second", max_change=2):
    swait.reset()  # remove any prior configuration
    swait.description.put("temperature")
    swait.channels.A.input_pv.put(swait.calculated_value.pvname)
    if setpoint is not None:
        swait.channels.A.input_value.put(setpoint)  # readback
        swait.channels.B.input_value.put(setpoint)  # setpoint
    swait.channels.C.input_value.put(noise)  # noise
    swait.channels.D.input_value.put(max_change)  # max_change
    swait.scanning_rate.put(rate)  # 1 second
    swait.calculation.put("A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)")

Setup our controller with a (randomly-selected) setpoint and scan rate.  Watch it start for a short time.

In [60]:
import random
import time

setup_temperature(controller, 10 + 30 * random.random(), rate="1 second")

t0 = time.time()
for i in range(10):
    time.sleep(1)
    print(
        f"{time.time() - t0:.2f}s:"
        f" readback={controller.calculated_value.get():.2f}"
        f" setpoint={controller.channels.B.input_value.get():.2f}"
    )

1.00s: readback=2.50 setpoint=38.38
2.00s: readback=4.02 setpoint=38.38
3.01s: readback=5.92 setpoint=38.38
4.01s: readback=7.42 setpoint=38.38
5.01s: readback=8.67 setpoint=38.38
6.02s: readback=9.86 setpoint=38.38
7.03s: readback=12.02 setpoint=38.38
8.03s: readback=14.33 setpoint=38.38
9.03s: readback=16.01 setpoint=38.38
10.03s: readback=19.01 setpoint=38.38


### temperature as a positioner

In [61]:
from apstools.devices import PVPositionerSoftDoneWithStop

temperature = PVPositionerSoftDoneWithStop(
    "",
    name="temperature",
    readback_pv=controller.calculated_value.pvname,
    setpoint_pv=controller.channels.B.input_value.pvname,
    tolerance=1,
)
temperature.wait_for_connection()
print(f"{temperature.position=}")

temperature.position=38.75128781233958


In [62]:
temperature.setpoint.put(temperature.position - 5)

t0 = time.time()
while not temperature.inposition:
    time.sleep(1)
    print(
        f"{time.time() - t0:.2f}s:"
        f" readback={temperature.position:.2f}"
        f" setpoint={temperature.setpoint.get():.2f}"
    )

1.00s: readback=36.91 setpoint=33.52
2.00s: readback=35.74 setpoint=33.52
3.01s: readback=33.06 setpoint=33.52


In [63]:
temperature.move(temperature.position + 3)

MoveStatus(done=True, pos=temperature, elapsed=1.7, success=True, settle_time=0.0)

In [64]:
temperature.move(25)

MoveStatus(done=True, pos=temperature, elapsed=4.6, success=True, settle_time=0.0)

### Configure taxi/fly to scan temperature

TODO